# Ukraine Refugee Mapping

In [ ]:
import json
import time
import requests

# google libraries
import googlemaps
import polyline

# mapping and shape utils
import folium
from folium import plugins

# data processing
import pandas as pd
import geopandas as gpd

import math

First, you need to enable the Google Directions API.

In [ ]:
gmaps = googlemaps.Client(key='YOUR KEY')

Add custom basemaps to folium

In [ ]:
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

## Read in Locations

In [ ]:
conflict_country="Honduras"

In [ ]:
df = pd.read_csv(f'inputs/{conflict_country}_locations.csv')
df.head(50)

In [ ]:
conflicts = df[df["location_type"]=="conflict_zone"]

In [ ]:
camps = df[df["location_type"]=="camp"]

In [ ]:
attractions = pd.read_csv('outputs/output_results.csv')

In [ ]:
attractions

In [ ]:
conflict_name=attractions["conflict"][0]

In [ ]:
def get_closest(loc_lat, loc_lon, targets, mode):
    chunk_size = 25
    list_targets = [targets[i:i+chunk_size] for i in range(0,targets.shape[0],chunk_size)]
    output = None
    closest_seconds = 100000000000
    closest_loc = None
    for i in list_targets:
        results = gmaps.distance_matrix(origins=[(loc_lat, loc_lon)],
                                        destinations=list(tuple(zip(i.latitude, i.longitude))), mode=mode)
        for idx, val in enumerate(results["rows"][0]["elements"]):
            if val["status"] == "ZERO_RESULTS":
                continue
            attraction = attractions[attractions["country"] == i.iloc[idx]["country"]].predicted_shares.iloc[0]
            seconds = val["duration"]["value"]*(1/math.sqrt(attraction))
            if seconds <= closest_seconds:
                closest_seconds = seconds
                closest_loc = i.iloc[idx]
                output = val
    return closest_loc, output

In [ ]:
conflict_exit_routes_transit = {}

In [ ]:
mode="transit"
for kk, conflict in conflicts.iterrows():
    if conflict["#name"] not in conflict_exit_routes_transit:
        closest_crossing, crossing_val = get_closest(conflict.latitude, conflict.longitude, camps, mode)
        if isinstance(closest_crossing, type(None)):
            print(f'{conflict["#name"]} No routes found')
        conflict_exit_routes_transit[conflict["#name"]] = dict(crossing=closest_crossing, 
                                                       crossing_v=crossing_val)

In [ ]:
conflict_exit_routes = {}

In [ ]:
conflict_exit_routes_transit[conflict["#name"]]["crossing"]

In [ ]:
NoneType = type(None)
mode="driving"
for kk, conflict in conflicts.iterrows():
    if isinstance(conflict_exit_routes_transit[conflict["#name"]]["crossing"], NoneType): 
        closest_crossing, crossing_val = get_closest(conflict.latitude, conflict.longitude, camps, mode)
        if isinstance(closest_crossing, type(None)):
            print(f'{conflict["#name"]} No routes found')
        conflict_exit_routes[conflict["#name"]] = dict(crossing=closest_crossing, 
                                                       crossing_v=crossing_val)

In [ ]:
conflict_exit_routes.keys()


In [ ]:
for kk, vv in conflict_exit_routes.items():
    if not isinstance(vv['crossing'], type(None)):
        vv['crossing'] = dict(vv['crossing'])
        
for kk, vv in conflict_exit_routes_transit.items():
    if not isinstance(vv['crossing'], type(None)):
        vv['crossing'] = dict(vv['crossing'])        

In [ ]:
import numpy as np

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

In [ ]:
with open(f'outputs/{conflict_name}_exit_routes_transit_hybrid.json','w') as f:
    f.write(json.dumps(conflict_exit_routes_transit))

In [ ]:
with open(f'outputs/{conflict_name}_exit_routes_driving_hybrid.json','w') as f:
    f.write(json.dumps(conflict_exit_routes, cls=NpEncoder))

In [ ]:
poland = 0
moldova = 0
for kk, vv in conflict_exit_routes_transit.items():
    try:
        if vv['crossing']['country'] == 'Poland':
            poland +=1
        if vv['crossing']['country'] == 'Moldova':
            moldova +=1            
        print(kk, vv['crossing']['#name'], vv['crossing']['country'])
    except:
        continue
print(poland,moldova)

## Directions and Plotting

In [ ]:
all_directions = {}
for kk, conflict in conflicts.iterrows():
    conflict_name = conflict['#name']
    print(f"Getting directions for conflict: {conflict_name}")
    
    if conflict_name in conflict_exit_routes:
        xing = conflict_exit_routes[conflict_name]['crossing']
        try:
            directions_result = gmaps.directions((conflict.latitude, conflict.longitude),
                                             (xing['latitude'], xing['longitude']),
                                             mode="driving")
            directions_result[0]['name']=xing['#name']
            directions_result[0]['country']=xing['country']
        except Exception as e:
            print(e)
            directions_result = None
        all_directions[conflict_name] = directions_result
    
all_directions_transit = {}
for kk, conflict in conflicts.iterrows():
    conflict_name = conflict['#name']
    print(f"Getting directions for conflict: {conflict_name}")
    
    xing = conflict_exit_routes_transit[conflict_name]['crossing']
    try:
        directions_result = gmaps.directions((conflict.latitude, conflict.longitude),
                                         (xing['latitude'], xing['longitude']),
                                         mode="transit")
        directions_result[0]['name']=xing['#name']
        directions_result[0]['country']=xing['country']
    except Exception as e:
        print(e)
        directions_result = None
    all_directions_transit[conflict_name] = directions_result    

In [ ]:
with open(f'outputs/{conflict_name}_border_crossing_directions_driving_hybrid.json','w') as f:
    f.write(json.dumps(all_directions))

with open(f'outputs/{conflict_name}_border_crossing_directions_transit_hybrid.json','w') as f:
    f.write(json.dumps(all_directions_transit))    

## Nearest Camp Map

Min/max normalization for population to obtain an appropriate line strkoe

In [ ]:
c_desc = conflicts.population.describe()

In [ ]:
def bucket_population(population):
    if population <= c_desc['25%']:
        stroke = 2.5
    elif population <= c_desc['50%']:
        stroke = 5
    elif population <= c_desc['75%']:
        stroke = 7.5
    else:
        stroke = 10
    return stroke
        

In [ ]:
conflicts['stroke'] = conflicts['population'].apply(lambda x: bucket_population(x))

In [ ]:
# Create Map
map = folium.Map(location=[conflicts.latitude.mean(),conflicts.longitude.mean()], zoom_start=6)

# Plot conflict starting points
for kk, start in conflicts.iterrows():
    start_m = folium.Marker([start.latitude, start.longitude], popup=start['#name'], 
                            icon=folium.Icon(icon='glyphicon glyphicon-fire', color='darkred'))
    start_m.add_to(map)
    
# Plot ending locations
for kk, vv in camps.iterrows():
    icon = 'glyphicon glyphicon-flag'
    color = 'orange'
    popup_text = f"<b>City Name: </b>{vv['#name']}<br>"\
                 f"<b>Country: </b>{vv['country']}"
    popup = folium.Popup(popup_text, max_width=300,min_width=150)
    xing = folium.Marker([vv.latitude, vv.longitude], popup=popup, 
                          icon=folium.Icon(icon=icon, color=color))
    xing.add_to(map)

# plot exit routes (driving)
fg_d = folium.FeatureGroup("Driving")
for kk, vv in all_directions.items():
    stroke = int(conflicts[conflicts['#name']==kk]['stroke'])
    population = "{:,}".format(int(conflicts[conflicts['#name']==kk]['population']))
    directions = all_directions[kk]
    if not isinstance(directions, type(None)):
        distance = directions[0]['legs'][0]['distance']['text']
        duration = directions[0]['legs'][0]['duration']['text']
        end_location = directions[0]['name']
        end_country = camps[camps['#name']==end_location].country.values[0]
        tooltip = f"Travel between <b>{kk}</b> and <b>{end_location}, {end_country}</b> by car is <b>"\
                  f"{distance}</b> and takes <b>{duration}</b>.</br></br>"\
                  f"<b>{population}</b> people are effected by this conflict."
        polyline_ = polyline.decode(directions[0]['overview_polyline']['points'])
        polyline_m = folium.PolyLine(polyline_, color='#4A89F3', tooltip=tooltip, weight=stroke)
        polyline_m.add_to(fg_d)
        
# plot exit routes (transit)
fg_t = folium.FeatureGroup("Transit")
for kk, vv in all_directions_transit.items():
    stroke = int(conflicts[conflicts['#name']==kk]['stroke'])
    population = "{:,}".format(int(conflicts[conflicts['#name']==kk]['population']))
    directions = all_directions_transit[kk]
    if not isinstance(directions, type(None)):
        if len(directions) > 0:
            distance = directions[0]['legs'][0]['distance']['text']
            duration = directions[0]['legs'][0]['duration']['text']
            end_location = directions[0]['name']
            end_country = camps[camps['#name']==end_location].country.values[0]
            tooltip = f"Travel between <b>{kk}</b> and <b>{end_location}, {end_country}</b> by transit is <b>"\
                      f"{distance}</b> and takes <b>{duration}</b>.</br></br>"\
                      f"<b>{population}</b> people are effected by this conflict."
            polyline_ = polyline.decode(directions[0]['overview_polyline']['points'])
            polyline_m = folium.PolyLine(polyline_, color='#7570b3', tooltip=tooltip, weight=stroke)
            polyline_m.add_to(fg_t)        
        
fg_d.add_to(map)
fg_t.add_to(map)
# folium.Choropleth(
#     manual_routes,
#     line_weight=3,
#     line_color='#f03b20',
#     name='Manually generated routes'
# ).add_to(map)
        
# Add custom basemaps
basemaps['Google Satellite Hybrid'].add_to(map)
# basemaps['Esri Satellite'].add_to(map)
# basemaps['Google Satellite'].add_to(map)
basemaps['Google Maps'].add_to(map)

# Add a layer control panel to the map.
map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(map)

In [ ]:
def add_legend(map):
    legend_html = """
    <style>
    @import url('https://fonts.googleapis.com/css2?family=Roboto:wght@100;300;400&display=swap');
    </style>
    
     <div style="
     padding-left:5px; padding-top:5px;
     position: fixed; 
     bottom: 50px; left: 50px; width: 160px; height: 120px;   
     border:2px solid grey; z-index:9999; border-radius: 15px;
     
     background-color:white;
     opacity: .85;
     
     font-size:14px;
     font-weight: bold;
     font-family: 'Roboto', sans-serif;
     ">

     <div class="awesome-marker-icon-darkred awesome-marker" style="margin-top: 10px; margin-left:5px;">
         <i class="fa-rotate-0 glyphicon glyphicon-glyphicon glyphicon-fire icon-white"></i>
     </div>
     <div style="margin-left:40px; margin-top:20px">Conflict Area</div>
     
     <div class="awesome-marker-icon-orange awesome-marker" style="margin-top: 60px; margin-left:5px;">
         <i class="fa-rotate-0 glyphicon glyphicon-glyphicon glyphicon-flag icon-white"></i>
     </div>
     <div style="margin-left:40px; margin-top:25px">Destination City</div>     
     
     
      </div> """.format( title = "Legend html")
    map.get_root().html.add_child(folium.Element( legend_html ))
    return map

In [ ]:
map = add_legend(map)

In [ ]:
display(map)

In [ ]:
# save map
map.save(f'{conflict_country}_Border_Crossing_Map_UPDATED.html')

In [ ]:
conflicts['#name'].unique()